In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Residual Block for ResNet
def residual_block(input_layer, filters, kernel_size, stride=1):
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, padding='same')(input_layer)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)

    conv2 = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(act1)
    bn2 = BatchNormalization()(conv2)

    # Shortcut connection
    if input_layer.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, strides=stride, padding='same')(input_layer)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = input_layer

    output = Add()([bn2, shortcut])
    output = Activation('relu')(output)
    return output

# Path to folder containing the Excel files
folder_path = "D:/OneDrive_2024-12-27/Segmented by seperating spindles/Balanced_segmented"

# Load and combine data from all Excel files
all_data = []
all_labels = []
for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        data = df.iloc[:, :-1].values  # Features (all columns except last)
        labels = df.iloc[:, -1].values  # Labels (last column)
        all_data.append(data)
        all_labels.append(labels)

# Combine all files into a single dataset
all_data = np.vstack(all_data)
all_labels = np.hstack(all_labels)

# Normalize the data
all_data = (all_data - np.min(all_data)) / (np.max(all_data) - np.min(all_data))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

# Reshape data for ResNet input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the ResNet model
input_layer = Input(shape=(X_train.shape[1], 1))
res_block1 = residual_block(input_layer, filters=32, kernel_size=3)
res_block2 = residual_block(res_block1, filters=64, kernel_size=3)
res_block3 = residual_block(res_block2, filters=128, kernel_size=3)

flatten = Flatten()(res_block3)
dense = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense)
output_layer = Dense(2, activation='softmax')(dropout)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
eval_metrics = model.evaluate(X_test, y_test, verbose=0)
print("\nModel Evaluation:")
print(f"Loss: {eval_metrics[0]:.4f}, Accuracy: {eval_metrics[1]:.4f}")

# Predict and print the confusion matrix
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Epoch 1/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 47s 59ms/step - accuracy: 0.8731 - loss: 0.7578 - val_accuracy: 0.9337 - val_loss: 0.1797
Epoch 2/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 60ms/step - accuracy: 0.9288 - loss: 0.2056 - val_accuracy: 0.8837 - val_loss: 0.3413
Epoch 3/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.9501 - loss: 0.1440 - val_accuracy: 0.9335 - val_loss: 0.1870
Epoch 4/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.9661 - loss: 0.0919 - val_accuracy: 0.7104 - val_loss: 1.1359
Epoch 5/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.9736 - loss: 0.0760 - val_accuracy: 0.8575 - val_loss: 0.3525
Epoch 6/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.9735 - loss: 0.0679 - val_accuracy: 0.6032 - val_loss: 2.2535
Epoch 7/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 60ms/step - accuracy: 0.9779 - loss: 0.0589 - val_accuracy: 0.6090 - val_loss: 3.0886
Epoch 8/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 42s 60ms/step - accuracy: 0.9823 - loss: 0.0428 - 